# Function Download
filename = Directory +"/"+ filename+".jpg" #อันนี้รองรับการสร้าง Folder แล้วยัดไปเรื่อยๆ 
ระวังเรื่องการใช้งาน Directory นะ

In [80]:
#!/usr/bin/python

# Note to Kagglers: This script will not run directly in Kaggle kernels. You
# need to download it and run it on your local machine.

# Downloads images from the Google Landmarks dataset using multiple threads.
# Images that already exist will not be downloaded again, so the script can
# resume a partially completed download. All images will be saved in the JPG
# format with 90% compression quality.

import sys, os, multiprocessing, csv
from PIL import Image
from io import BytesIO
from urllib.request import urlopen
import time
import os

def ParseData(data_file):
    csvfile = open(data_file, 'r')
    csvreader = csv.reader(csvfile)
    key_url_list = [line[:3] for line in csvreader]
    return key_url_list[1:]  # Chop off header

import urllib.request

def download_image(url,filename,Directory):
#*************************************** ระวังผิด directory นะ set ก่อนด้วย ****************************
        filename = Directory +"/"+ filename+".jpg" #อันนี้รองรับการสร้าง Folder แล้วยัดไปเรื่อยๆ
#*************************************** ระวังผิด directory นะ set ก่อนด้วย ****************************
        if os.path.exists(filename):
            print('Image %s already exists. Skipping download.' % filename)
            return
        try:
            response = urlopen(url)
            image_data = response.read()
        except:
            print('Warning: Could not download image %s from %s' % (key, url))
            return
        try:
            pil_image = Image.open(BytesIO(image_data))
        except:
            print('Warning: Failed to parse image %s' % key)
            return
        urllib.request.urlretrieve(url,filename)     


# Create downloadable file
fomat data เป็น Csv. 3 colurm จะเป็นผมให้มันเป็น แรก 3 line ในคำสั่ง ParseData(path)

In [81]:
csv_download = ParseData('landmark_recognition-master/data/data.csv')

In [82]:
#Play safe for index_boundary tranform list to dict 
label_to_category = ParseData('landmark_recognition-master/data/label_to_category.csv')
label_to_category_dict = {}

for el_lable in  label_to_category :
    label_to_category_dict[el_lable[2]] = [el_lable[0],el_lable[1]]
print(label_to_category_dict["0"]) 

['192931', 'york_river_state_park']


# Data split train-test

In [83]:
import numpy as np 
import pandas as pd
len(csv_download)

91618

In [84]:
#Split data 
#ตัวของข้อมูลนั้นทุกตัวคือคนละ Class 
#Expect ต้องแยกได้ 7 Folder

lst_raw_order = [] #L คือ List ของภาพที่เป็นคนละ Class
for i in range(len(csv_download)) :  # Loop นี้จะแยกและหาตำแหน่ง สุดท้าย ของ Class แต่ละ Class
    if i == len(csv_download)-1 :
        lst_raw_order.append(int(csv_download[i][0]))
        break
    if (csv_download[i][2] != csv_download[i+1][2]) :
        #print(i)
        lst_raw_order.append(int(csv_download[i][0]))
c = 0
# for j in lst_data : 
#     print(c,j,"\n")
#     c += 1 

In [85]:
raw_data = np.asarray(csv_download)
lst_raw_data_split = []
for i in range(100) : 
    if ( i == 0 ) : 
        lst_raw_data_split.append(raw_data[:lst_raw_order[i]])
    else : 
        lst_raw_data_split.append(raw_data[lst_raw_order[i-1]:lst_raw_order[i]] )
print(len(lst_raw_data_split))
#test Miss data in list
for j in range(100) : 
    for el_lst in range(1,len(lst_raw_data_split[i])) : 
        if (lst_raw_data_split[i][el_lst][2] != lst_raw_data_split[i][el_lst][2] ) : 
            print("error_detect",lst_raw_data_split[1][el_lst])

100


In [105]:
train_set = []
test_set  = []
var_set   = []

for el_class in lst_raw_data_split :
    el_class = pd.DataFrame(el_class)
    train, validate, test = np.split(el_class.sample(frac=1), [int(.8*len(el_class)), int(.9*len(el_class))])
    train_set.append(train)
    test_set.append(test)
    var_set.append(validate)
    Data.append(list([train,test,validate]))

In [106]:
Data[0] #Train

[         0                                                  1  2
 2467  2467  https://upload.wikimedia.org/wikipedia/commons...  0
 2171  2171  https://upload.wikimedia.org/wikipedia/commons...  0
 1701  1701  https://upload.wikimedia.org/wikipedia/commons...  0
 1048  1048  https://upload.wikimedia.org/wikipedia/commons...  0
 1968  1968  https://upload.wikimedia.org/wikipedia/commons...  0
 1315  1315  https://upload.wikimedia.org/wikipedia/commons...  0
 498    498  https://upload.wikimedia.org/wikipedia/commons...  0
 1932  1932  https://upload.wikimedia.org/wikipedia/commons...  0
 452    452  https://upload.wikimedia.org/wikipedia/commons...  0
 224    224  https://upload.wikimedia.org/wikipedia/commons...  0
 715    715  https://upload.wikimedia.org/wikipedia/commons...  0
 1339  1339  https://upload.wikimedia.org/wikipedia/commons...  0
 2504  2504  https://upload.wikimedia.org/wikipedia/commons...  0
 1533  1533  https://upload.wikimedia.org/wikipedia/commons...  0
 309    30

In [107]:
Data[1] # test

[         0                                                  1  2
 1141  1141  https://upload.wikimedia.org/wikipedia/commons...  0
 360    360  https://upload.wikimedia.org/wikipedia/commons...  0
 2540  2540  https://upload.wikimedia.org/wikipedia/commons...  0
 450    450  https://upload.wikimedia.org/wikipedia/commons...  0
 1949  1949  https://upload.wikimedia.org/wikipedia/commons...  0
 580    580  https://upload.wikimedia.org/wikipedia/commons...  0
 1255  1255  https://upload.wikimedia.org/wikipedia/commons...  0
 2008  2008  https://upload.wikimedia.org/wikipedia/commons...  0
 1848  1848  https://upload.wikimedia.org/wikipedia/commons...  0
 1357  1357  https://upload.wikimedia.org/wikipedia/commons...  0
 1205  1205  https://upload.wikimedia.org/wikipedia/commons...  0
 899    899  https://upload.wikimedia.org/wikipedia/commons...  0
 417    417  https://upload.wikimedia.org/wikipedia/commons...  0
 117    117  https://upload.wikimedia.org/wikipedia/commons...  0
 615    61

In [116]:
Data[2] #Validation

[         0                                                  1  2
 880    880  https://upload.wikimedia.org/wikipedia/commons...  0
 197    197  https://upload.wikimedia.org/wikipedia/commons...  0
 1109  1109  https://upload.wikimedia.org/wikipedia/commons...  0
 2620  2620  https://upload.wikimedia.org/wikipedia/commons...  0
 1975  1975  https://upload.wikimedia.org/wikipedia/commons...  0
 1429  1429  https://upload.wikimedia.org/wikipedia/commons...  0
 541    541  https://upload.wikimedia.org/wikipedia/commons...  0
 1861  1861  https://upload.wikimedia.org/wikipedia/commons...  0
 1276  1276  https://upload.wikimedia.org/wikipedia/commons...  0
 1337  1337  https://upload.wikimedia.org/wikipedia/commons...  0
 400    400  https://upload.wikimedia.org/wikipedia/commons...  0
 418    418  https://upload.wikimedia.org/wikipedia/commons...  0
 512    512  https://upload.wikimedia.org/wikipedia/commons...  0
 43      43  https://upload.wikimedia.org/wikipedia/commons...  0
 2373  237

# Start Download

แปลงเป็น Function ที่สามารถเรียกค่า ของ Directory และจัดการไฟล์

In [174]:
label_to_category_dict

{'0': ['192931', 'york_river_state_park'],
 '1': ['186080', 'osaka_prefectural_flower_garden'],
 '2': ['19605', 'saiful_muluk_lake'],
 '3': ['171683', 'lok_virsa_museum,_islamabad'],
 '4': ['189811', 'lake_como'],
 '5': ['86869', 'mount_rainier_national_park'],
 '6': ['90021', 'rohtas_fort'],
 '7': ['62798', 'enchanted_floral_gardens_of_kula'],
 '8': ['181631', 'leesylvania_state_park'],
 '9': ['10618', 'botanical_garden_jevremovac'],
 '10': ['165596', 'edinburgh_castle'],
 '11': ['165900', 'mount_arapiles'],
 '12': ['145015', 'texas_state_library_and_archives_commission'],
 '13': ['171772', 'university_of_chicago_library'],
 '14': ['67416', 'manassas_national_battlefield_park'],
 '15': ['177194', 'royal_national_park'],
 '16': ['107750', 'chippokes_plantation_state_park'],
 '17': ['137203', 'port_of_casablanca'],
 '18': ['73300', 'iguazu_falls'],
 '19': ['149980', 'khotyn_fortress'],
 '20': ['176018', 'hayravank_monastery'],
 '21': ['184313', 'kahanu_garden'],
 '22': ['16658', 'glasne

In [186]:

#*************************************** ระวังผิด directory นะ set ก่อนด้วย ****************************
 #Use disc directory
#*************************************** ระวังผิด directory นะ set ก่อนด้วย ****************************
start_time = time.time()
temp_dir = 0 
directory = label_to_category_dict[str(temp_dir)][1] # label_to_category_dict[str(temp_dir)][1] จะเรียก ชื่อ Landmark จาก Dict มา
for i in range(3) :
    if ( i == 0  ) : 
        base_directory = "D:/Train"
        if not (os.path.exists(base_directory)) :
            os.makedirs(base_directory)
    elif (i == 1 ) : 
        base_directory = "D:/Test"
        if not (os.path.exists(base_directory)) :
            os.makedirs(base_directory)
    elif (i == 2 ) : 
        base_directory = "D:/Validate"
        if not (os.path.exists(base_directory)) :
            os.makedirs(base_directory)
    else : 
        print("error create Test-Train-Valiadate")
        break
    for j in range(len(Data[0])): 
        print("Do CLass 1")
        for k in range(len(Data[j][i][0]))  :
            if os.path.exists(base_directory +'/'+ directory): #ใส่รูปไปเรื่อยๆ จนกว่าจะไปเจอ Directory 
                #print('Image %s already exists. Skipping download.' % base_directory + directory)
                download_image(Data[i][j][1].iloc[k],Data[i][j][0].iloc[k],base_directory+'/'+ directory)
            else : 
                os.makedirs(base_directory +'/'+ directory) #เมื่อต้องสร้างไฟล์ใหม่ ต้องมีการใส่รูป
                download_image(Data[i][j][1].iloc[k],Data[i][j][0].iloc[k],base_directory+'/'+ directory)
            if ( k % 150 == 0) : 
                print("Pass",str(k))
        
end_time = time.time()
print(end_time-start_time)

Do CLass 1
Image D:/Train/york_river_state_park/2467.jpg already exists. Skipping download.
Pass 0
Image D:/Train/york_river_state_park/2171.jpg already exists. Skipping download.
Image D:/Train/york_river_state_park/1701.jpg already exists. Skipping download.
Image D:/Train/york_river_state_park/1048.jpg already exists. Skipping download.
Image D:/Train/york_river_state_park/1968.jpg already exists. Skipping download.
Image D:/Train/york_river_state_park/1315.jpg already exists. Skipping download.
Image D:/Train/york_river_state_park/498.jpg already exists. Skipping download.
Image D:/Train/york_river_state_park/1932.jpg already exists. Skipping download.
Image D:/Train/york_river_state_park/452.jpg already exists. Skipping download.
Image D:/Train/york_river_state_park/224.jpg already exists. Skipping download.
Image D:/Train/york_river_state_park/715.jpg already exists. Skipping download.
Image D:/Train/york_river_state_park/1339.jpg already exists. Skipping download.
Image D:/Train

KeyboardInterrupt: 

# Test Case 1 
ตัวของข้อมูลนั้นทุกตัวคือคนละ Class 
Expect ต้องแยกได้ 7 Folder

# https://stackoverflow.com/questions/273192/how-can-i-safely-create-a-nested-directory

In [110]:
#Split data 
# Test Case 1 
#ตัวของข้อมูลนั้นทุกตัวคือคนละ Class 
#Expect ต้องแยกได้ 7 Folder

l = [] #L คือ List ของภาพที่เป็นคนละ Class
for i in range(10000) :  # Loop นี้จะแยกและหาตำแหน่ง สุดท้าย ของ Class แต่ละ Class
    if i == 9999 :
        break
    if (csv_download[i][2] != csv_download[i+1][2]) :
        #print(i)
        l.append(csv_download[i])
for j in l : 
    print(*j,"\n")

2626 https://upload.wikimedia.org/wikipedia/commons/c/cf/YR_Estuaries_Day_2015_%2820226742864%29.jpg 0 

3463 http://upload.wikimedia.org/wikipedia/commons/b/bc/Cyrtanthus_mackenii3.jpg 1 

4007 https://upload.wikimedia.org/wikipedia/commons/9/98/Saif_ul_Mulook.jpg 2 

5054 https://upload.wikimedia.org/wikipedia/commons/a/a4/%40ibneAzhar-Heritage_Museum_-Islamabad-Pakistan_%2834%29.JPG 3 

5730 https://upload.wikimedia.org/wikipedia/commons/5/55/Lake_Como%2C_Wikimania_2016%2C_MP_019.jpg 4 

7080 https://upload.wikimedia.org/wikipedia/commons/7/71/Mount_Rainier_National_Park_%289287685657%29.jpg 5 

7594 https://upload.wikimedia.org/wikipedia/commons/5/54/Rohtas_trip.jpg 6 



In [111]:
#Download data
#อันนี้จะทำการ 7 Folder ที่มีรูปคนละรูป ในแต่ละ Folder มี 1 รูป
start_time = time.time()
#*************************************** ระวังผิด directory นะ set ก่อนด้วย ****************************
base_directory = "D:/" #Use disc directory
#*************************************** ระวังผิด directory นะ set ก่อนด้วย ****************************
temp_dir = 0 
directory = label_to_category_dict[str(temp_dir)][1] # label_to_category_dict[str(temp_dir)][1] จะเรียก ชื่อ Landmark จาก Dict มา
for i in range(len(l)):
    if os.path.exists(base_directory + directory):
        #print('Image %s already exists. Skipping download.' % base_directory + directory)
        download_image(l[i][1],l[i][0],base_directory+ directory)
    else : 
        os.makedirs(base_directory + directory)
        download_image(l[i][1],l[i][0],base_directory+ directory)
    if i == len(l)-1 :
        temp_dir += 1
        directory = label_to_category_dict[str(temp_dir)][1]
    elif (l[i][2] != l[i+1][2]) :
        temp_dir += 1
        directory = label_to_category_dict[str(temp_dir)][1]
        
    
    
end_time = time.time()
print(end_time-start_time)

6.829430341720581
